# Torchtext Tutorial

## 1. 데이터 다운로드

### 1-0. 라이브러리 임포트

In [3]:
import urllib.request
import pandas as pd

### 1-1. IMDB 리뷰 데이터 다운로드

In [12]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7fe4add8f4d0>)

### 1-2. 데이터 불러오기

In [13]:
df=pd.read_csv('./data/IMDb_Reviews.csv',encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [14]:
print('전체 샘플의 갯수: {}'.format(len(df)))

전체 샘플의 갯수: 50000


### 1-3. Train/Test Split

In [15]:
train_df=df[:25000]
test_df=df[25000:]

In [16]:
# train/test csv 파일 저장
# index=False로 하면 인덱스를 저장하지 않습니다.
train_df.to_csv('./data/train_data.csv',index=False)
test_df.to_csv('./data/test_data.csv',index=False)

## 2. 필드 정의하기

### 2-0. torchtext 라이브러리 임포트

In [9]:
from torchtext.legacy import data

### 2-1. Field 정의하기

In [11]:
TEXT=data.Field(
    sequential=True,
    use_vocab=True,
    tokenize=str.split,
    lower=True,
    batch_first=True,
    fix_length=20
)

LABEL=data.Field(
    sequential=False,
    use_vocab=False,
    batch_first=False,
    is_target=True
)

## 3. 데이터셋 만들기

### 3-1. torchtext 라이브러리 임포트

In [17]:
from torchtext.legacy.data import TabularDataset

### 3-2. TabularDataset을 이용해 데이터셋 만들기
- path : 파일 경로
- train/test : train,test 파일 이름
- format : 파일의 형태 (csv,tsv,json)
- fields : 필드 정보
- skip_header : 데이터의 첫번째 줄은 무시 (csv 파일의 경우 feature명)

In [19]:
train_data,test_data=TabularDataset.splits(
    path='./data',
    train='train_data.csv',test='test_data.csv',
    format='csv',
    fields=[('text',TEXT),('label',LABEL)],
    skip_header=True
    )

In [20]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

훈련 샘플의 개수 : 25000
테스트 샘플의 개수 : 25000


In [21]:
# vars() 함수를 통해 주어진 인덱스의 샘플을 확인 할 수 있다.
print(vars(train_data[0]))

{'text': ['my', 'family', 'and', 'i', 'normally', 'do', 'not', 'watch', 'local', 'movies', 'for', 'the', 'simple', 'reason', 'that', 'they', 'are', 'poorly', 'made,', 'they', 'lack', 'the', 'depth,', 'and', 'just', 'not', 'worth', 'our', 'time.<br', '/><br', '/>the', 'trailer', 'of', '"nasaan', 'ka', 'man"', 'caught', 'my', 'attention,', 'my', 'daughter', 'in', "law's", 'and', "daughter's", 'so', 'we', 'took', 'time', 'out', 'to', 'watch', 'it', 'this', 'afternoon.', 'the', 'movie', 'exceeded', 'our', 'expectations.', 'the', 'cinematography', 'was', 'very', 'good,', 'the', 'story', 'beautiful', 'and', 'the', 'acting', 'awesome.', 'jericho', 'rosales', 'was', 'really', 'very', 'good,', "so's", 'claudine', 'barretto.', 'the', 'fact', 'that', 'i', 'despised', 'diether', 'ocampo', 'proves', 'he', 'was', 'effective', 'at', 'his', 'role.', 'i', 'have', 'never', 'been', 'this', 'touched,', 'moved', 'and', 'affected', 'by', 'a', 'local', 'movie', 'before.', 'imagine', 'a', 'cynic', 'like', 'me

In [23]:
# 필드 구성 확인
print(train_data.fields.items())

dict_items([('text', <torchtext.legacy.data.field.Field object at 0x7fe4add8f090>), ('label', <torchtext.legacy.data.field.Field object at 0x7fe4add8f050>)])


## 4. 단어집합 만들기
- 토큰화 전처리가 끝난 후 각 단어에 고유한 정수를 맵핑해주는 정수 인코딩(Integer Encoding) 작업이 진행되어야 합니다 그 전에 각 단어에 고유한 정수를 부여하기 위해서는 단어 집합을 만들어주어야 합니다
- 단어 집합 생성 : <필드명>.build_vocab()

In [24]:
# train_data에 TEXT필드에 있는 것들을 단어집합으로 만들꺼야
# 단어 집합에는 최소 10번이상 등장한 단어들을 넣어야 하고 단어집의 최대 크기는 10000이야
TEXT.build_vocab(train_data,min_freq=10,max_size=10000)

In [25]:
# Q) max_size를 10000개로 설정했는데 왜 10002?
# A) <unk>,<pad> 와 같은 special token이 추가되었기 떄문입니다.
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

단어 집합의 크기 : 10002


- 생성된 단어의 집합은 "<필드명>.vocab.stoi" 를 통해서 확인이 가능합니다.

In [ ]:
print(TEXT.vocab.stoi)

## 5. torchtext를 활용한 데이터로더 만들기
- 데이터로더의 역할 : 데이터로더는 데이터셋에서 미니배치만큼 데이터를 로드하게 만들어주는 역할을 합니다.
- torchtext에서는 iterator를 사용하여 데이터로더를 생성합니다.

### 5-0. 라이브러리 임포트

In [29]:
from torchtext.legacy.data import Iterator

In [31]:
batch_size=10
train_loader=Iterator(dataset=train_data,batch_size=batch_size)
test_loader=Iterator(dataset=test_data,batch_size=batch_size)

In [32]:
# 25000 / 10 = 2500
print('훈련 데이터의 미니배치의 갯수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니배치의 갯수 : {}'.format(len(test_loader)))

훈련 데이터의 미니배치의 갯수 : 2500
테스트 데이터의 미니배치의 갯수 : 2500


In [33]:
batch=next(iter(train_loader))

- 일반적인 데이터로더의 경우 미니 배치를 텐서로 가져오지만 torchtext의 데이터로더는 torchtext.legacy.batch.Batch 객체로 가져오게 됩니다. 실제 데이터 텐서에 접근하기 위해서는 정확한 필드명을 사용해야 합니다.

In [34]:
print(type(batch))

<class 'torchtext.legacy.data.batch.Batch'>


In [36]:
print(batch.text)

tensor([[  29,   48,  251,  114,    3,   25,  369,   38,   96,   43,    0, 1648,
            8,   43,    0,    0, 1321,   47,   15,    3],
        [   9,   61,  465,   10,  664,   34, 3871,    0,    2,  119,  850,    9,
           91,  207,   63,    6, 1654,   11,   17,    7],
        [  10,   20,    5,    0,  279,   14, 3943,   31,    2, 1039, 1228,  864,
          273,    2,  366,    0,   11,  608,   24,    2],
        [  44,  291,  268,   16,   32,  238,  211, 2771,   10,  212,   28,    2,
           30,   16,  916,   43,  731,    4,   29,  290],
        [1422,   10,   20,   14, 4333,    9,   83,   98,   11,   14,    2,  119,
           20,  124,   95,   18,   11,  280,   14, 1546],
        [   0,   11,  362,  126,   59,  996, 4771,  535,    4,   60, 1157,    3,
         6866, 1162, 5675,  428,    7,    3, 4925, 2416],
        [   9,  310,   42,   10,   20,    8,    3, 4748,    4,    9,   14,    0,
            3, 1743,   35,   30,  359, 1035, 1045, 1693],
        [   2,  320,   16, 